In [ ]:
# 如果不想改程式碼，直接下載成.py檔或直接複製程式碼
# 0. 到conda promot 下載需要的套件 pip install numpy; pip install cv2; pip install ultralytics
# 1. 把訓練好的 .pt 檔放到跟你執行這份程式碼同一個資料夾(或使用預訓練好的yolov8模型 例如: model = YOLO("yolov8n.pt")， 根據電腦效能，可以換成其他模型
# 2. 放入你的即時影像網址
# 3. 如果想將影片儲存，# video_writer_result可以調整
# 4. 按下 q 可以跳出循環
# 學校電腦確定能跑，怕筆電炸掉就用學校電腦跑

In [ ]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

# 載入 YOLO 模型
model = YOLO("best.pt")

# 即時影像網址
video_url = "https://cctv1.kctmc.nat.gov.tw/6e559e58/"
# video_url = 'https://cctv6.kctmc.nat.gov.tw/f6007b0d/'

# 打開cap
cap = cv2.VideoCapture(video_url)

if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

# 獲取影片width, height, fps
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# 存儲軌跡歷史
track_history = defaultdict(lambda: [])
# video_writer_result = cv2.VideoWriter('C:/Users/JJ/Pictures/test/output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# 判斷是否開啟
while cap.isOpened():
    # 對每幀讀取
    success, frame = cap.read()

    if success:
        # 對每幀進行 YOLO 追蹤
        results = model.track(frame, persist=True, show_labels=False, show_conf=False, save_txt=True)

        # 確保檢測結果有效
        if results[0].boxes.id is not None and len(results[0].boxes) > 0:
            # 獲取框和跟蹤 ID
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # 可視化結果
            frame_vis = results[0].plot()

            # 繪製軌跡
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))
                if len(track) > 30:
                    track.pop(0)

                # 繪製跟蹤線條
                points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame_vis, [points], isClosed=False, color=(151, 255, 255), thickness=8)

            # 顯示
            cv2.imshow("frame", frame_vis)

            # 如果按下 q ，跳出循環
        if cv2.waitKey(1) & 0xFF == ord("q"):
           cap.release()
           cv2.destroyAllWindows()
           break
        else:
            # 如果沒有檢測到任何物體，直接寫入原畫面
            cv2.imshow("frame", frame)
    else:
        # 如果讀取失敗，則跳出循環
        break

cap.release()
# video_writer_result.release()
cv2.destroyAllWindows()